In [1]:
import time
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# chromedriver PATH
chromedriver_path = '../chromedriver'
service = Service(chromedriver_path)
chrome_options = Options()
chrome_options.add_argument("--disable-images")
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
driver.get("https://www.google.com/maps/place/Cervecer%C3%ADa+Sierra/@40.4400515,-3.7104325,15z/data=!4m8!3m7!1s0xd42285d9fe6f033:0xe27e4de86263432a!8m2!3d40.4342037!4d-3.7104844!9m1!1b1!16s%2Fg%2F11b6d43yfv?entry=ttu&g_ep=EgoyMDI0MDkxNi4wIKXMDSoASAFQAw%3D%3D")
wait = WebDriverWait(driver, 5)

try:
    accept_button = driver.find_element(By.XPATH, "//button[@aria-label='Aceptar todo']")
    accept_button.click()
    print("Se hizo clic en 'Aceptar todo'")
    time.sleep(1)  # Wait
except Exception as e:
    # If button doesnt appear, continue
    pass

In [ ]:
# Wait until reviews page is open
wait = WebDriverWait(driver, 10)
reviews_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')))

raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')

print(raw_html_reviews[0].text)

raw_html_stars_resumme = raw_html_reviews[0].find_elements(By.CSS_SELECTOR, 'tr.BHOKXe')

ratings = []
reviews_counts = []

for star in raw_html_stars_resumme:
    rating_text = star.get_attribute('aria-label')
    rating_parts = rating_text.split(',')  # Dividimos la información por coma
    stars = rating_parts[0].split()[0]  # Extraemos el número de estrellas
    num_reviews = rating_parts[1].strip().split()[0].replace('.', '')  # Extraemos el número de reseñas y eliminamos el punto
    ratings.append(int(stars))
    reviews_counts.append(int(num_reviews))

stars_resumme = pd.DataFrame({'stars': ratings, 'reviews': reviews_counts})
display(stars_resumme)

In [ ]:
# Asumiendo que raw_html_reviews es una lista de WebElements
html = raw_html_reviews[0].get_attribute('outerHTML')
print(html)

In [ ]:
local_guide_info = raw_html_reviews[0].find_element(By.CLASS_NAME, "RfDO5c").text
print("Local Guide Info:", local_guide_info)


In [ ]:
from selenium.webdriver.common.by import By
import time

# Inicializa las listas para almacenar los datos
reviewers = []
ratings = []
review_texts = []
review_dates = []
local_guides = []
text_backups = []

# Inicializa un conjunto para rastrear las reseñas procesadas
processed_review_ids = set()

# Scroll loop
SCROLL_PAUSE_TIME = 120

last_height = driver.execute_script("return arguments[0].scrollHeight", reviews_container)
while True:
    # Extract all visible reviews
    raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
    
    # Extract data for each review
    for review in raw_html_reviews:
        try:
            # Obtener el 'data-review-id' de la reseña
            review_id = review.get_attribute('data-review-id')
            
            # Verificar si la reseña ya ha sido procesada
            if review_id in processed_review_ids:
                continue  # Omitir reseñas ya procesadas
            
            # Agregar el ID de la reseña al conjunto de procesados
            processed_review_ids.add(review_id)
            
            # Click on "See more" button to open the review
            try:
                more_buttons = review.find_elements(By.CSS_SELECTOR, 'button.w8nwRe')
                for button in more_buttons:
                    if button.is_displayed():
                        button.click()
                        time.sleep(1)
            except Exception as e:
                print(f"No se encontró un botón 'Ver más' en esta reseña: {str(e)}")

            try: reviewer = review.find_element(By.CSS_SELECTOR, 'div.d4r55').text
            except: reviewer = ''

            try: review_text = review.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text
            except: review_text = ''

            try: rating = review.find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label')
            except: rating = ''

            try: review_date = review.find_element(By.CSS_SELECTOR, 'span.rsqaWe').text
            except: review_date = ''

            try: local_guide = review.find_element(By.CLASS_NAME, "RfnDt").text
            except: local_guide = ''

            print(reviewer + ' - ' + rating + ' - ' + review_date + ' - ' + review_text) 

            reviewers.append(reviewer)
            review_texts.append(review_text)
            ratings.append(rating)
            review_dates.append(review_date)
            local_guides.append(local_guide)

            text_backups.append(review.text)

        except Exception as e:
            print(f"Error al extraer una reseña: {str(e)}")

    # Scroll down the page
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", reviews_container)
    time.sleep(10)

    new_height = driver.execute_script("return arguments[0].scrollHeight", reviews_container)
    print(new_height)
    print(last_height)
    # If height doesn't change, wait to load more reviews.
    if new_height == last_height:
        print('No more reviews detected, forcing additional scroll')
        # Wait more reviews to be loaded
        time.sleep(SCROLL_PAUSE_TIME) 
        new_raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
        if len(new_raw_html_reviews) == len(raw_html_reviews): 
            print('No more reviews loaded, exiting scroll')
            break
    else:
        last_height = new_height


In [ ]:
collected_data = pd.DataFrame({
    'author': reviewers,
    'local_guide_info': local_guides,
    'rating': ratings,
    'review': review_texts,
    'date': review_dates,
    'price': prices,
    'food': comidas,
    'service': servicios,
    'atmosphere': ambientes,
    'recommendations': recomendados,
    'text_backup': text_backups
})
display(collected_data)

In [ ]:
collected_data.tail(30)

In [ ]:
csv_file_path = '../data/raw/collected_data.csv'
collected_data.to_csv(csv_file_path, index=False)


In [ ]:
- comprobar para otros establecimientos

In [ ]:
sample = collected_data.iloc[20].text_backup
sample

In [ ]:
import re

clean_text = re.sub(r'\\ue[0-9a-f]{3}', '', sample)  # Eliminar emojis (\ue...)
clean_text = re.sub(r'\n+', '\n', clean_text)  # Eliminar saltos de línea excesivos
clean_text = clean_text.strip()  # Eliminar espacios al principio y final

# Extraer el precio por persona
precio_persona = re.search(r'Precio por persona\n([0-9€\- ]+)', clean_text)
precio_persona = precio_persona.group(1) if precio_persona else ''

# Extraer valores de Comida, Servicio y Ambiente
comida = re.search(r'Comida: (\d+)', clean_text)
comida = comida.group(1) if comida else ''

servicio = re.search(r'Servicio: (\d+)', clean_text)
servicio = servicio.group(1) if servicio else ''

ambiente = re.search(r'Ambiente: (\d+)', clean_text)
ambiente = ambiente.group(1) if ambiente else ''

# Mostrar los resultados
print(f'Precio por persona: {precio_persona}')
print(f'Comida: {comida}')
print(f'Servicio: {servicio}')
print(f'Ambiente: {ambiente}')